# Exploring LA
### Charlotte Huang
### Dec 2019
This notebook is for the capstone project of the IBM Data Science Certificate. <br>
Los Angeles is one of the most popular cities in the US with diverse population and plenty of social places. More and more people are moving to or vising LA. They may be facing big difficulties of choosing neighbohoods. This project aims to cluster all neighborhoods in LA according to their venues and provide people with some suggestions. 

## Import Packages

In [1]:
!pip install geopy

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [2]:
!pip install folium

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [3]:
!pip install yellowbrick

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests

from scipy.spatial.distance import cdist

print('Libraries imported.')

Libraries imported.


/Users/huangyihuan/anaconda2/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


## 1. Download and Explore Geo-Spatial Dataset

In [5]:
with open('la_city.json') as f:
  laCity = json.load(f)

In [6]:
laCity = laCity["features"]

In [7]:
df = pd.DataFrame(columns = ["Neighbourhood","Latitude", "Longitude"])
for data in laCity:
    lat = np.array(data["geometry"]["coordinates"][0][0]).mean(axis = 0)[1]
    lon = np.array(data["geometry"]["coordinates"][0][0]).mean(axis = 0)[0]
    name = data["properties"]["name"]
    df = df.append({"Neighbourhood": name, "Latitude": lat, "Longitude": lon}, ignore_index=True)

In [8]:
print df.shape
df.head()

(114, 3)


,Neighbourhood,Latitude,Longitude
0,Adams-Normandie,34.030702,-118.298988
1,Arleta,34.244105,-118.430287
2,Arlington Heights,34.042849,-118.320744
3,Atwater Village,34.130468,-118.265232
4,Baldwin Hills/Crenshaw,34.010793,-118.358400


## 2. Visualize map data

In [9]:
!pip install geocoder

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [11]:
address = 'Los Angeles'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


In [12]:
# create map of Downtown Toronto using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

## 3. Clustering and Segmenting

### Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'S4JSAPBEB2HWJ1T2HPR3BJX1PVGDIAA30G5C1151VRNGCRZX' # your Foursquare ID
CLIENT_SECRET = 'KMM3DCK4WLTODUAOILM2K0ZCSFLDXM3B3DA52QLU0BPTAPKA' # your Foursquare Secret
VERSION = '20191009' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S4JSAPBEB2HWJ1T2HPR3BJX1PVGDIAA30G5C1151VRNGCRZX
CLIENT_SECRET:KMM3DCK4WLTODUAOILM2K0ZCSFLDXM3B3DA52QLU0BPTAPKA


### Explore neighborhoods in LA

In [14]:
# Get API from foursquares and transform them into dataframes
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
LIMIT = 100
la_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Adams-Normandie
Arleta
Arlington Heights
Atwater Village
Baldwin Hills/Crenshaw
Bel-Air
Beverly Crest
Beverly Grove
Beverlywood
Boyle Heights
Brentwood
Broadway-Manchester
Canoga Park
Carthay
Central-Alameda
Century City
Chatsworth
Chatsworth Reservoir
Chesterfield Square
Cheviot Hills
Chinatown
Cypress Park
Del Rey
Downtown
Eagle Rock
East Hollywood
Echo Park
El Sereno
Elysian Park
Elysian Valley
Encino
Exposition Park
Fairfax
Florence
Glassell Park
Gramercy Park
Granada Hills
Green Meadows
Griffith Park
Hancock Park
Hansen Dam
Harbor City
Harbor Gateway
Harvard Heights
Harvard Park
Highland Park
Historic South-Central
Hollywood
Hollywood Hills
Hollywood Hills West
Hyde Park
Jefferson Park
Koreatown
Lake Balboa
Lake View Terrace
Larchmont
Leimert Park
Lincoln Heights
Los Feliz
Manchester Square
Mar Vista
Mid-City
Mid-Wilshire
Mission Hills
Montecito Heights
Mount Washington
North Hills
North Hollywood
Northridge
Pacific Palisades
Pacoima
Palms
Panorama City
Pico-Robertson
Pico-Union
P

In [16]:
la_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams-Normandie,34.030702,-118.298988,Orange Door Sushi,34.032485,-118.299368,Sushi Restaurant
1,Adams-Normandie,34.030702,-118.298988,Shell,34.033095,-118.300025,Gas Station
2,Adams-Normandie,34.030702,-118.298988,Sushi Delight,34.032445,-118.299525,Sushi Restaurant
3,Adams-Normandie,34.030702,-118.298988,Little Xian,34.032292,-118.299465,Sushi Restaurant
4,Adams-Normandie,34.030702,-118.298988,Little Xian Fusion Restaurant,34.032485,-118.299368,Sushi Restaurant


### Analyze each neighborhood

In [17]:
# one hot encoding
la_onehot = pd.get_dummies(la_venues['Venue Category'], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot['Neighborhood'] = la_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = list(la_onehot.columns)
cols.insert(0, cols.pop(cols.index('Neighborhood')))
la_onehot = la_onehot[cols]

la_onehot.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Trail,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General College & University,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightlife Spot,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Rest Area,Restaurant,River,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Ukrainian Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Watch Shop,Water Park,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,W

In [18]:
la_grouped = la_onehot.groupby('Neighborhood').mean().reset_index()
la_grouped.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Trail,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General College & University,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightlife Spot,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Rest Area,Restaurant,River,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Ukrainian Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Watch Shop,Water Park,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,W

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = la_grouped['Neighborhood']

for ind in np.arange(la_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams-Normandie,Sushi Restaurant,Taco Place,Park,Grocery Store,Gas Station,Yoga Studio,Playground,Fast Food Restaurant,Farmers Market,Farm
1,Arleta,Convenience Store,Video Store,Historic Site,Bakery,Farm,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant
2,Arlington Heights,Donut Shop,Grocery Store,Rental Car Location,Art Gallery,Liquor Store,Restaurant,Fabric Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
3,Atwater Village,Mexican Restaurant,Taco Place,Convenience Store,Gym / Fitness Center,Buffet,Mobile Phone Shop,Miscellaneous Shop,Breakfast Spot,Fast Food Restaurant,Electronics Store
4,Baldwin Hills/Crenshaw,Shop & Service,Clothing Store,Yoga Studio,Drugstore,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm


### Clustering

In [23]:
k = 6

In [24]:
la_grouped_clustering = la_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(la_grouped_clustering)

In [25]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

la_merged = df.drop([40, 69, 91], 0)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
la_merged = la_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')


la_merged.head() # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams-Normandie,34.030702,-118.298988,5,Sushi Restaurant,Taco Place,Park,Grocery Store,Gas Station,Yoga Studio,Playground,Fast Food Restaurant,Farmers Market,Farm
1,Arleta,34.244105,-118.430287,5,Convenience Store,Video Store,Historic Site,Bakery,Farm,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant
2,Arlington Heights,34.042849,-118.320744,4,Donut Shop,Grocery Store,Rental Car Location,Art Gallery,Liquor Store,Restaurant,Fabric Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
3,Atwater Village,34.130468,-118.265232,4,Mexican Restaurant,Taco Place,Convenience Store,Gym / Fitness Center,Buffet,Mobile Phone Shop,Miscellaneous Shop,Breakfast Spot,Fast Food Restaurant,Electronics Store
4,Baldwin Hills/Crenshaw,34.010793,-118.358400,0,Shop & Service,Clothing Store,Yoga Studio,Drugstore,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm


In [26]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged['Latitude'], la_merged['Longitude'], la_merged['Neighbourhood'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius= 3,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
la_merged.loc[la_merged['Cluster Labels'] == 0, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Baldwin Hills/Crenshaw,Shop & Service,Clothing Store,Yoga Studio,Drugstore,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm
17,Chatsworth Reservoir,Clothing Store,Yoga Studio,Drugstore,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
89,Sun Valley,Golf Driving Range,Clothing Store,Athletics & Sports,Yoga Studio,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Farm


In [28]:
la_merged.loc[la_merged['Cluster Labels'] == 1, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Beverly Crest,Juice Bar,Lawyer,Clothing Store,Park,Farm,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant
8,Beverlywood,Business Service,Lawyer,Park,Boutique,Food Court,Food,Flower Shop,Flea Market,Filipino Restaurant,Fast Food Restaurant
13,Carthay,Yoga Studio,Movie Theater,Gym / Fitness Center,Dentist's Office,Farmers Market,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm
30,Encino,Scenic Lookout,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farmers Market,Donut Shop
38,Griffith Park,Lawyer,Park,Trail,Rest Area,Flea Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm,Flower Shop
49,Hollywood Hills West,Yoga Studio,Other Great Outdoors,Art Gallery,Farmers Market,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant
51,Jefferson Park,Park,Fried Chicken Joint,Sandwich Place,Mexican Restaurant,Carpet Store,Flea Market,Filipino Restaurant,Fast Food Restaurant,Flower Shop,Drugstore
63,Mission Hills,Gym / Fitness Center,Park,Church,Auto Workshop,Athletics & Sports,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop
64,Montecito Heights,Park,Lake,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Yoga Studio
77,Porter Ranch,Park,Water Park,Brazilian Restaurant,Nightlife Spot,Lawyer,Flea Market,Filipino Restaurant,Flower Shop,Fast Food Restaurant,Food


In [29]:
la_merged.loc[la_merged['Cluster Labels'] == 2, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Harvard Park,Park,Art Gallery,Yoga Studio,Farmers Market,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm
86,South Park,Park,Yoga Studio,Farmers Market,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant
106,West Hills,Park,Yoga Studio,Farmers Market,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant


In [30]:
la_merged.loc[la_merged['Cluster Labels'] == 3, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Lake View Terrace,Farm,Yoga Studio,Farmers Market,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Drugstore


In [31]:
la_merged.loc[la_merged['Cluster Labels'] == 4, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Arlington Heights,Donut Shop,Grocery Store,Rental Car Location,Art Gallery,Liquor Store,Restaurant,Fabric Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
3,Atwater Village,Mexican Restaurant,Taco Place,Convenience Store,Gym / Fitness Center,Buffet,Mobile Phone Shop,Miscellaneous Shop,Breakfast Spot,Fast Food Restaurant,Electronics Store
7,Beverly Grove,Clothing Store,Boutique,Lounge,Department Store,Lingerie Store,Spa,Japanese Restaurant,Cosmetics Shop,Hotel,Sandwich Place
9,Boyle Heights,Fried Chicken Joint,Pizza Place,Theater,Mexican Restaurant,Yoga Studio,Farmers Market,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant
12,Canoga Park,Cosmetics Shop,Pet Store,Marijuana Dispensary,Tennis Court,Chinese Restaurant,Park,Restaurant,Health & Beauty Service,Furniture / Home Store,Mexican Restaurant
14,Central-Alameda,Light Rail Station,Food,Gas Station,Mexican Restaurant,Farm,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant
15,Century City,Coffee Shop,Food Truck,Mexican Restaurant,Restaurant,Gym,Salad Place,Movie Theater,Café,Hotel,Park
18,Chesterfield Square,Ice Cream Shop,Shoe Store,Chinese Restaurant,Grocery Store,BBQ Joint,Clothing Store,Video Game Store,Hardware Store,Pharmacy,Mobile Phone Shop
20,Chinatown,Chinese Restaurant,Vietnamese Restaurant,Bakery,Taiwanese Restaurant,Coffee Shop,Bar,Brewery,Sandwich Place,Tea Room,Fried Chicken Joint
21,Cypress Park,Park,Mexican Restaurant,Discount Store,Bakery,Farm,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant


In [32]:
la_merged.loc[la_merged['Cluster Labels'] == 5, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams-Normandie,Sushi Restaurant,Taco Place,Park,Grocery Store,Gas Station,Yoga Studio,Playground,Fast Food Restaurant,Farmers Market,Farm
1,Arleta,Convenience Store,Video Store,Historic Site,Bakery,Farm,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant
5,Bel-Air,Home Service,Dive Bar,Yoga Studio,Farm,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farmers Market
10,Brentwood,Spa,Home Service,Yoga Studio,Farmers Market,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm
11,Broadway-Manchester,Office,Food,Motel,Gym,Comedy Club,Farm,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop
16,Chatsworth,Movie Theater,Flower Shop,Gourmet Shop,Trail,Food,Flea Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Drugstore
19,Cheviot Hills,Gym,Yoga Studio,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farmers Market
35,Gramercy Park,Moving Target,Convenience Store,Discount Store,Farmers Market,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant
36,Granada Hills,Construction & Landscaping,Jewelry Store,Yoga Studio,Farmers Market,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farm
39,Hancock Park,Poke Place,Bookstore,Sushi Restaurant,Health & Beauty Service,Farm,Eastern European Restaurant,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant
